In [1]:
import pennylane as qml
from pennylane import numpy as np
 
# Define the molecule
symbols  = ['H', 'H', 'H', 'H']
r_bohr = 1.8897259886
coordinates = np.array([[0.0,0.0, 0.0], [0.0, 0.0, 3.0*r_bohr], [0.0,0.0,6.0*r_bohr],[0.0, 0.0, 9.0*r_bohr]])  # Coordinates of your molecule
 
electrons = 4
charge = 0
active_orbitals = 4 #Spatial orbitals
active_electrons = 4
 
# Build the electronic Hamiltonian
H, qubits = qml.qchem.molecular_hamiltonian(symbols, coordinates, basis = "sto-6g", method="pyscf",active_electrons=electrons, active_orbitals=active_orbitals)
 
# Define the HF state
hf_state = qml.qchem.hf_state(active_electrons, qubits)
 
# Generate single and double excitations
singles, doubles = qml.qchem.excitations(active_electrons, qubits)
 
# Map excitations to the wires the UCCSD circuit will act on
s_wires, d_wires = qml.qchem.excitations_to_wires(singles, doubles)
 
# Define the device
dev = qml.device("lightning.qubit", wires=qubits)
 
# Define the qnode
@qml.qnode(dev)
def circuit(params, wires, s_wires, d_wires, hf_state):
    qml.UCCSD(params, wires, s_wires, d_wires, hf_state)
    return qml.expval(H)
 
# Define the initial values of the circuit parameters
params = np.zeros(len(singles) + len(doubles))
 
# Define the optimizer
optimizer = qml.GradientDescentOptimizer(stepsize=0.5)
 
# Optimize the circuit parameters and compute the energy
for n in range(500):
    params, energy = optimizer.step_and_cost(circuit, params,
    wires=range(qubits), s_wires=s_wires, d_wires=d_wires, hf_state=hf_state)
    if n % 2 == 0:
        print("step = {:},  E = {:.8f} Ha".format(n, energy))
 

step = 0,  E = -1.33256889 Ha
step = 2,  E = -1.50179164 Ha
step = 4,  E = -1.63902084 Ha
step = 6,  E = -1.73499424 Ha
step = 8,  E = -1.79570565 Ha
step = 10,  E = -1.83209949 Ha
step = 12,  E = -1.85346816 Ha
step = 14,  E = -1.86599187 Ha
step = 16,  E = -1.87338620 Ha
step = 18,  E = -1.87780132 Ha
step = 20,  E = -1.88047067 Ha
step = 22,  E = -1.88210498 Ha
step = 24,  E = -1.88311797 Ha
step = 26,  E = -1.88375337 Ha
step = 28,  E = -1.88415653 Ha
step = 30,  E = -1.88441518 Ha
step = 32,  E = -1.88458292 Ha
step = 34,  E = -1.88469283 Ha
step = 36,  E = -1.88476557 Ha
step = 38,  E = -1.88481415 Ha
step = 40,  E = -1.88484690 Ha
step = 42,  E = -1.88486916 Ha
step = 44,  E = -1.88488440 Ha
step = 46,  E = -1.88489491 Ha
step = 48,  E = -1.88490221 Ha
step = 50,  E = -1.88490730 Ha
step = 52,  E = -1.88491088 Ha
step = 54,  E = -1.88491340 Ha
step = 56,  E = -1.88491519 Ha
step = 58,  E = -1.88491646 Ha
step = 60,  E = -1.88491736 Ha
step = 62,  E = -1.88491801 Ha
step = 64,  E

In [2]:
## VQE - Exact
from qiskit_nature.units import DistanceUnit
from qiskit_nature.second_q.drivers import PySCFDriver

from qiskit_nature.second_q.mappers import JordanWignerMapper
from qiskit_algorithms import MinimumEigensolver
from qiskit.quantum_info import SparsePauliOp

from qiskit_nature.second_q.operators import SparseLabelOp
from qiskit_nature.second_q.mappers import QubitMapper
from qiskit_nature.second_q.problems import BaseProblem
from qiskit_nature.second_q.problems import EigenstateResult


from qiskit_algorithms import VQE
from qiskit_algorithms.optimizers import SPSA, SLSQP, COBYLA, QNSPSA, GradientDescent, L_BFGS_B, POWELL
from qiskit.primitives import Estimator
from qiskit_nature.second_q.circuit.library import HartreeFock, UCCSD
from qiskit_nature.second_q.operators import FermionicOp
from qiskit_nature.units import DistanceUnit
from qiskit_nature.second_q.drivers import PySCFDriver
from qiskit_nature.second_q.mappers import JordanWignerMapper, QubitMapper

from qiskit_nature.second_q.algorithms import GroundStateEigensolver
from qiskit_nature.second_q.circuit.library import HartreeFock, UCCSD
from qiskit_nature.second_q.mappers import ParityMapper
import numpy as np
from pyscf import gto, scf, mcscf
from qiskit_algorithms import AdaptVQE, AdaptVQEResult




H4 = 'H 0 0 0.0; H 0 0 3.0; H 0 0 6.0; H 0 0 9.0'

n_it = 1
# Specify other molecular properties
charge = 0
spin = 0
basis = 'sto6g'


# Using driver to get fermionic Hamiltonian
# PySCF example
from qiskit.primitives import Estimator
it1 = []
energies_exact= []




optimizer = POWELL(maxiter=1e6, xtol=1e-12, maxfev=1e6)
mapper = JordanWignerMapper()

for i in range(n_it):
        print('The iteration now is ', i)
        driver =  PySCFDriver(atom=H4,
                      unit=DistanceUnit.ANGSTROM,
                      charge=charge,
                      spin=spin,
                      basis=basis)
        qmolecule = driver.run()
        fermionic_op = qmolecule.hamiltonian.second_q_op()
        qubit_op = mapper.map(fermionic_op)
        ansatz = UCCSD(
                qmolecule.num_spatial_orbitals,
                qmolecule.num_particles,
                mapper,
                initial_state = HartreeFock(
                        qmolecule.num_spatial_orbitals,
                        qmolecule.num_particles,
                        mapper,
                        )
                        )
        
        estimator = Estimator()
        vqe = VQE(estimator, ansatz, optimizer, initial_point= [0.0]* ansatz.num_parameters)
        vqe_calcC = vqe.compute_minimum_eigenvalue(qubit_op)
        vqe_result = qmolecule.interpret(vqe_calcC).total_energies[0].real
        energies_exact.append(vqe_result)
        it1.append(i)


print('The angles are',it1)
print("Energies are", energies_exact)
print('VQE Result is',vqe_calcC)


The iteration now is  0


/var/folders/tg/76l0rq6n6m3bzwcmn22dk8_59wwtc3/T/ipykernel_38409/602815023.py:76: DeprecationWarning: The class ``qiskit.primitives.estimator.Estimator`` is deprecated as of qiskit 1.2. It will be removed no earlier than 3 months after the release date. All implementations of the `BaseEstimatorV1` interface have been deprecated in favor of their V2 counterparts. The V2 alternative for the `Estimator` class is `StatevectorEstimator`.
  estimator = Estimator()


KeyboardInterrupt: 